# Ingest ADS Notebook Files (Interactive)

![files_to_database](files_to_database.png)

This Notebook will load selected files (from a local or networked file system) into a SQL Server database.

This notebook is designed run _interactively_ and does not require any supporting objects in eltSnap. 

The associated notebook: [Ingest ADS Notebook Files (Project)](ingest_ads_notebook_files_project.ipynb) can be used to Deploy this notebook capability as a runnable _Project_ into the eltSnap database.

> For best results, verify/update the **Notebook Configuration** code cell below, and then use the '**Run all**' option for this Notebook.

## Notebook Configuration

In [ ]:
# Configuration Parameters
$FolderLocation = 'C:\ADS\Repository\'
$ServerInstance = 'localhost'
$Database = 'eltsnap_v2'

# Set Begin Time
$BegTime = $(Get-Date)
'Beg Time: ' +  $BegTime

# Show Environment
'Run Values:'
'Folder Location: ' + $FolderLocation
'Server: ' + $ServerInstance
'Database: ' +$Database

## Reset staging table

In [ ]:
Invoke-Sqlcmd -ServerInstance $ServerInstance -Database $Database -Query "TRUNCATE TABLE [stg].[files]"

## Load Notebook Files into table: [stg].[files]


In [ ]:
Get-ChildItem -Path $FolderLocation -Recurse -Include *.ipynb | Select-Object @{l="FullName";e={($_.FullName).Replace($FolderLocation,"")}}, BaseName, @{l="Extension";e={($_.Extension).Replace(".","")}}, LastWriteTime, Length, @{l="Contents";e={(Get-Content $_ -Raw).ToString()}} | Write-SqlTableData -ServerInstance $ServerInstance -DatabaseName $database -SchemaName "stg" -TableName "files"

## Load .SQL Files into table: [stg].[files]

In [ ]:
Get-ChildItem -Path $FolderLocation -Recurse -Include *.sql | Select-Object @{l="FullName";e={($_.FullName).Replace($FolderLocation,"")}}, BaseName, @{l="Extension";e={($_.Extension).Replace(".","")}}, LastWriteTime, Length, @{l="Contents";e={(Get-Content $_ -Raw).ToString()}} | Write-SqlTableData -ServerInstance $ServerInstance -DatabaseName $database -SchemaName "stg" -TableName "files"

## Merge table: [stg].[files] into: [ads].[files]

In [ ]:
Invoke-Sqlcmd -ServerInstance $ServerInstance -Database $Database -Query "EXEC [ads].[Merge Staged Files];"

## Ingest all Notebooks Files into table: [ads].[notebooks]

In [ ]:
Invoke-Sqlcmd -ServerInstance $ServerInstance -Database $Database -Query "EXEC [ads].[Ingest All Notebooks];"

## Run Summary

In [ ]:
'Beg Time: ' +  $BegTime

$EndTime = $(Get-Date)
'End Time: ' +  $EndTime

$EndTime - $BegTime
